# Observer

Implementation 

In [1]:
class IObserver{
    public:
    virtual ~IObserver() {}
    virtual void update(double temp, double humidity, double pressure) = 0;
};

In [2]:
class ISubject {
    public:
    virtual ~ISubject() {}
    virtual void registerObserver(IObserver& observer) = 0;
    virtual void removeObserver(IObserver& observer)  = 0;
    virtual void notifyObservers()  = 0;
};

In [3]:
class IDisplayElement{
    public:
    virtual ~IDisplayElement() {}
    virtual void display() const = 0;
};

Now we are good to define conreate class

In [4]:
class CWeatherData:public ISubject{
    private:
    std::list<IObserver*> observers;
    double temperature;
    double humidity;
    double pressure;


    public:

    CWeatherData(){}

    virtual ~CWeatherData() override {}

    virtual void registerObserver(IObserver& observer) override
    {
        observers.push_back(&observer);
    }

    void removeObserver(IObserver& observer) override
    {
        observers.remove(&observer);
    }

    void notifyObservers() override
    {
        for (IObserver* i: observers)
        {
            i->update(temperature, humidity, pressure);
        }
    }

    void measurementsChanged()
    {
        notifyObservers();
    }

    //For testbed
    void setMeasurement(double temperature, double humidity, double pressure)
    {
        this->temperature = temperature;
        this->humidity = humidity;
        this->pressure = pressure;
        measurementsChanged();
    }

};

In [7]:
class CCurrentConditionDisplay: public IObserver, public IDisplayElement
{
private:
    ISubject& m_subject;
    double m_temp;
    double m_humidity;
    double m_pressure;

public:
    CCurrentConditionDisplay(ISubject& subject ):m_subject(subject)
    {
        m_subject.registerObserver(*this);
    }

    virtual ~CCurrentConditionDisplay() final override {}

    virtual void update(double temp, double humidity, double pressure) override
    {
        m_temp = temp;
        m_humidity = humidity;
        m_pressure = pressure;

        display();
    }

    virtual void display() const override
    {
        printf("Temperature: %.2f",m_temp);
        printf("Humidity: %.2f",m_humidity);
        printf("Pressure: %.2f",m_pressure);
    }

};

In [8]:
class CStatisticDisplay: public IObserver, public IDisplayElement
{
private:
    ISubject& m_subject;
    double m_avgTemp;
    double m_avgHumidity;
    double m_avgPressure;
    uint count;

public:
    CStatisticDisplay(ISubject& subject):m_subject(subject), m_avgTemp(0), m_avgHumidity(0), m_avgPressure(0), count(0)
    {
        m_subject.registerObserver(*this);
    }

    virtual ~CStatisticDisplay() final override {}

    virtual void update(double temp, double humidity, double pressure) override
    {
        ++count;
        m_avgTemp = (m_avgTemp + temp)/count;
        m_avgHumidity = (m_avgHumidity + humidity) / count;
        m_avgPressure = (m_avgPressure + pressure) / count;

        display();
    }

    virtual void display() const override
    {
        printf("Average Temperature: %.2f",m_avgTemp);
        printf("Average Humidity: %.2f",m_avgHumidity);
        printf("Average Pressure: %.2f",m_avgPressure);
    }

};

In [9]:
    CWeatherData weatherData;

    CCurrentConditionDisplay currentCondition(weatherData);

    CStatisticDisplay statisticDisplay(weatherData);

    weatherData.setMeasurement(30,50,20);
    weatherData.setMeasurement(35,48,19);
